In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-int-extrap)

We estimate $\displaystyle\int_0^2 x^2 e^{-2x}\, dx$ using extrapolation. First we use `quadgk` to get an accurate value.

In [2]:
using QuadGK
f = x -> x^2 * exp(-2x);
a = 0;
b = 2;
Q, _ = quadgk(f, a, b, atol=1e-14, rtol=1e-14)
@show Q;

Q = 0.1904741736116139

We start with the trapezoid formula on $n=N$ nodes.

In [3]:
N = 20;       # the coarsest formula
n = N;
h = (b - a) / n;
t = h * (0:n);
y = f.(t);

We can now apply weights to get the estimate $T_f(N)$.

In [4]:
T = [h * (sum(y[2:n]) + y[1] / 2 + y[n+1] / 2)]

1-element Vector{Float64}:
 0.19041144993926784

Now we double to $n=2N$, but we only need to evaluate $f$ at every other interior node and apply {eq}`nc-doubling`.

In [5]:
n = 2n;
h = h / 2;
t = h * (0:n);
T = [T; T[end] / 2 + h * sum(f.(t[2:2:n]))]

2-element Vector{Float64}:
 0.19041144993926784
 0.19045880585951175

We can repeat the same code to double $n$ again.

In [6]:
n = 2n;
h = h / 2;
t = h * (0:n);
T = [T; T[end] / 2 + h * sum(f.(t[2:2:n]))]

3-element Vector{Float64}:
 0.19041144993926784
 0.19045880585951175
 0.1904703513046443

Let us now do the first level of extrapolation to get results from Simpson's formula. We combine the elements `T[i]` and `T[i+1]` the same way for $i=1$ and $i=2$.

In [7]:
S = [(4T[i+1] - T[i]) / 3 for i in 1:2]

2-element Vector{Float64}:
 0.19047459116625973
 0.19047419978635513

With the two Simpson values $S_f(N)$ and $S_f(2N)$ in hand, we can do one more level of extrapolation to get a sixth-order accurate result.

In [8]:
R = (16S[2] - S[1]) / 15

0.1904741736943615

We can make a triangular table of the errors:
```{tip}
:class: dropdown
The value `nothing` equals nothing except `nothing`.
```

In [9]:
err = [T .- Q [nothing; S .- Q] [nothing; nothing; R - Q]]
@pt :header=["order 2", "order 4", "order 6"] err

order 2,order 4,order 6
-6.27237e-5,nothing,nothing
-1.53678e-5,4.17555e-7,nothing
-3.82231e-6,2.61747e-8,8.27476e-11


If we consider the computational time to be dominated by evaluations of $f$, then we have obtained a result with about twice as many accurate digits as the best trapezoid result, at virtually no extra cost.